In [3]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv("env_variables.env")
client = OpenAI()

In [4]:
from promptrefiner import AbstractLLM, PromptTrackerClass, OpenaiCommunicator

In [5]:
###
class LlamaCPPModel(AbstractLLM):
    def __init__(self, base_url, api_key, temperature=0.1, max_tokens=200):
        super().__init__()
        self.temperature = temperature
        self.max_tokens = max_tokens
        from openai import OpenAI
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        
    def predict(self, input_text, system_prompt):
        response = self.client.chat.completions.create(
        model="mistral",
        messages=[
            {"role": "system", "content": system_prompt},  # Update this as per your needs
            {"role": "user", "content": input_text}
        ],
        temperature=self.temperature,
        max_tokens=self.max_tokens,
    )
        llm_response = response.choices[0].message.content
        return llm_response

In [6]:
llamamodel = LlamaCPPModel(base_url="http://localhost:8000/v1", api_key="sk-xxx", temperature=0.1, max_tokens=400)

In [7]:
###
output_evaluation_1 = """{
    "John": {"cousin": "Mark", "wife": "Lisa"},
    "Lisa": {"husband": "John", "best friend": "Tina"},
    "Mark": {"cousin": "John", "sister": "Tina"},
    "Tina": {"best friend": "Lisa", "brother": "Mark"}
}"""

output_evaluation_2 = """{
    "Sarah": {"role": "manager", "assistant": "Tom", "meeting with": ["Tom", "Emily", "Alex"]},
    "Tom": {"role": "assistant", "manager": "Sarah", "meeting with": ["Sarah", "Emily", "Alex"]},
    "Emily": {"role": "project leader", "meeting with": ["Sarah", "Tom", "Alex"], "mentor": "Alex"},
    "Alex": {"role": "CEO", "meeting with": ["Sarah", "Tom", "Emily"], "mentee": "Emily"}
}"""

output_evaluation_3 = """{
    "Kevin": {"neighbor": "Lucas", "son": "Mike", "played with": "Lucas"},
    "Lucas": {"neighbor": "Kevin", "daughter": "Jenny", "played with": "Kevin"},
    "Jenny": {"father": "Lucas", "business partner": "Mike", "cousin": "Tim"},
    "Mike": {"father": "Kevin", "business partner": "Jenny", "friend": "Tim"},
    "Tim": {"cousin": "Jenny", "friend": "Mike"}
}"""

In [8]:
input_evaluations = [input_evaluation_1, input_evaluation_2, input_evaluation_3]
output_evaluations = [output_evaluation_1, output_evaluation_2, output_evaluation_3]

In [9]:
###
init_sys_prompt = """You are an AI that receives an input text about relatiosnhips between people and returns their relatioships as a python dictionary"""
promptTracker = PromptTrackerClass(init_system_prompt = init_sys_prompt)
promptTracker.add_evaluation_examples(input_evaluations, output_evaluations)
#promptTracker.add_evaluation_examples(input_evaluation_2, output_evaluation_2)

In [10]:
promptTracker.run_initial_prompt(llm_model=llamamodel)

In [ ]:
print(promptTracker.llm_responses[0][0])

In [18]:
openai_communicator = OpenaiCommunicator(client=client, openai_model_code="gpt-4-0125-preview")

In [19]:
openai_communicator.refine_system_prompt(prompttracker=promptTracker, llm_model=llamamodel, number_of_iterations=3)

In [ ]:
print(promptTracker.llm_responses[3][2])

In [ ]:
print(promptTracker.llm_system_prompts[3])